In [1]:
import spacy
import pandas as pd
import en_core_web_lg
from stanfordcorenlp import StanfordCoreNLP
import json
from collections import defaultdict
import nltk

<img src="diagram.png" style="height: 50px; width: 1000px;"/>

Methods investigated Flowchart

In [2]:
starwars_text = 'Darth Vader, also known by his birth name Anakin Skywalker, is a fictional character in the Star Wars franchise. Darth Vader appears in the original film trilogy as a pivotal antagonist whose actions drive the plot, while his past as Anakin Skywalker and the story of his corruption are central to the narrative of the prequel trilogy. The character was created by George Lucas and has been portrayed by numerous actors. His appearances span the first six Star Wars films, as well as Rogue One, and his character is heavily referenced in Star Wars: The Force Awakens. He is also an important character in the Star Wars expanded universe of television series, video games, novels, literature and comic books. Originally a Jedi who was prophesied to bring balance to the Force, he falls to the dark side of the Force and serves the evil Galactic Empire at the right hand of his Sith master, Emperor Palpatine (also known as Darth Sidious).'
starwars_text

'Darth Vader, also known by his birth name Anakin Skywalker, is a fictional character in the Star Wars franchise. Darth Vader appears in the original film trilogy as a pivotal antagonist whose actions drive the plot, while his past as Anakin Skywalker and the story of his corruption are central to the narrative of the prequel trilogy. The character was created by George Lucas and has been portrayed by numerous actors. His appearances span the first six Star Wars films, as well as Rogue One, and his character is heavily referenced in Star Wars: The Force Awakens. He is also an important character in the Star Wars expanded universe of television series, video games, novels, literature and comic books. Originally a Jedi who was prophesied to bring balance to the Force, he falls to the dark side of the Force and serves the evil Galactic Empire at the right hand of his Sith master, Emperor Palpatine (also known as Darth Sidious).'

In [ ]:
starwars_text = '''Iron Man is a fictional superhero appearing in American comic books published by Marvel Comics. The character was co-created by writer and editor Stan Lee, developed by scripter Larry Lieber, and designed by artists Don Heck and Jack Kirby. The character made his first appearance in Tales of Suspense #39 (cover dated March 1963), and received his own title in Iron Man #1 (May 1968). Also in 1963, the character founded the Avengers alongside Thor, Ant-Man, Wasp and the Hulk. A wealthy American business magnate, playboy, philanthropist, inventor and ingenious scientist, Anthony Edward "Tony" Stark suffers a severe chest injury during a kidnapping. When his captors attempt to force him to build a weapon of mass destruction, he instead creates a mechanized suit of armor to save his life and escape captivity. Later, Stark develops his suit, adding weapons and other technological devices he designed through his company, Stark Industries. He uses the suit and successive versions to protect the world as Iron Man. Although at first concealing his true identity, Stark eventually publicly reveals himself to be Iron Man. Initially, Iron Man was a vehicle for Stan Lee to explore Cold War themes, particularly the role of American technology and industry in the fight against communism. Subsequent re-imaginings of Iron Man have transitioned from Cold War motifs to contemporary matters of the time. Throughout most of the character's publication history, Iron Man has been a founding member of the superhero team the Avengers and has been featured in several incarnations of his own various comic book series. Iron Man has been adapted for several animated TV shows and films. The Marvel Cinematic Universe character was portrayed by Robert Downey Jr. in the Marvel Cinematic Universe films Iron Man (2008), The Incredible Hulk (2008) in a cameo, Iron Man 2 (2010), The Avengers (2012), Iron Man 3 (2013), Avengers: Age of Ultron (2015), Captain America: Civil War (2016), Spider-Man: Homecoming (2017), Avengers: Infinity War (2018) and Avengers: Endgame (2019). The character also appeared in Spider-Man: Far From Home (2019) and in the upcoming Black Widow (2021) through archive footage.'''
starwars_text

> Note: 6 sentences

# SpaCy NER

In [16]:
! python -m spacy download en_core_web_lg

     |████████████████████████████████| 782.7 MB 6.8 MB/s eta 0:00:011     |██████████████▎                 | 350.2 MB 62.7 MB/s eta 0:00:07     |██████████████████▉             | 461.9 MB 5.7 MB/s eta 0:00:56
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [18]:

# nlp = en_core_web_lg.load()
nlp = spacy.load("en_core_web_lg")
# doc = nlp(starwars_text)

# ner_dict = {}
# for x in doc.ents:
#     ner_dict[x.text] = x.label_
# ner_dict

ModuleNotFoundError: No module named 'transformers.tokenization_gpt2'

# Stanford NER

> Note: Does not perform well at all compared to SpaCy NER. Able to recognize PERSONs but not in partial fragments. Not able to recognize entities other than LOCATION or PERSON such as WORK_OF_ART or DATE, etc

In [ ]:
# # NLTK
# # sentences = nltk.sent_tokenize(starwars_text)

# spaCy
run_this=0
if run_this==1:
    nlp = spacy.lang.en.English()
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    doc = nlp(starwars_text)
    sentences = [sent.string.strip() for sent in doc.sents]

    ner_tagger = nltk.tag.StanfordNERTagger("./stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz", "./stanford-ner-2018-10-16/stanford-ner.jar")

    ner_dict = {}
    results = []

    nlp = spacy.lang.en.English()
    tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)
    for sent in sentences:
        words = [token.orth_ for token in tokenizer(sent)]
        print(words)
        tagged = ner_tagger.tag(words)
        results += tagged

    for res in results:
        ner_dict[res[0]] = res[1]

In [ ]:
ner_dict

# Coreference Resolution

## TODO: Look into Neural Coref from Huggingface: https://github.com/huggingface/neuralcoref

Look into CoreNLP base model for Python
https://stanfordnlp.github.io/CoreNLP/coref.html

> fixed root permission error: https://github.com/Lynten/stanford-corenlp/issues/26

### Generate coreferences and dependencies

In [ ]:
nlp = StanfordCoreNLP("./stanford-corenlp-4.2.0", quiet=False)
annotated = nlp.annotate(starwars_text, properties={'annotators': 'coref', 'pipelineLanguage': 'en'})
result = json.loads(annotated)

In [ ]:
corefs = result['corefs']
corefs

### Resolve coreferences

In [ ]:
corefs = result['corefs']
print("Coreferences found: ",len(corefs))
print("Named entities: " , ner_dict.keys())

In [ ]:
replace_coref_with = []
sentence_wise_replacements = defaultdict(list) 
sentence_wise_replacements

In [ ]:
sentences = nltk.sent_tokenize(starwars_text)
# nlp = spacy.lang.en.English()
# nlp.add_pipe(nlp.create_pipe('sentencizer'))
# doc = nlp(starwars_text)
# sentences = [sent.string.strip() for sent in doc.sents]

print('Number of sentences: ', len(sentences))
print(sentences)

> Note: Here, nltk sentence tokenizer is more accurate than spaCy

### Organize corefs and replacements

In [ ]:
for index,coreferences in enumerate(corefs.values()):
    replace_with = coreferences[0]
    for reference in coreferences:
        if reference["text"] in ner_dict.keys() or reference["text"][reference["headIndex"]-reference["startIndex"]] in ner_dict.keys():
            replace_with = reference
        sentence_wise_replacements[reference["sentNum"]-1].append((reference,index))
    replace_coref_with.append(replace_with["text"])  

sentence_wise_replacements[0].sort(key=lambda tup: tup[0]["startIndex"]) 

### Pronoun Replacement with Named Entity

In [ ]:
# nlp = spacy.lang.en.English()
# tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)
tokenizer = nltk.word_tokenize

In [ ]:
#Carry out replacement
for index,sent in enumerate(sentences):
    replacement_list = sentence_wise_replacements[index]    
    for item in replacement_list[::-1]:                     
        to_replace = item[0]                                
        replace_with = replace_coref_with[item[1]]
        replaced_sent = ""
        words = tokenizer(sent)
        
        
        for i in range(len(words)-1,to_replace["endIndex"]-2,-1):
            replaced_sent = words[i] + " " + replaced_sent
        
        replaced_sent = replace_with + " " + replaced_sent
        
        for i in range(to_replace["startIndex"]-2,-1,-1):
            replaced_sent = words[i] + " " + replaced_sent
        
        #print(replaced_sent)
        sentences[index] = replaced_sent

result = ""
for sent in sentences:
    result += sent

### Original Text

In [ ]:
starwars_text

### New Text

TODO: fix the weird spacing

In [ ]:
result

# Relation Extraction of using Stanford OpenIE

Subject removal -> not URI (Literals)

In [ ]:
from openie import StanfordOpenIE

triples = []
with StanfordOpenIE() as client:
    for triple in client.annotate(result):
        triples.append(triple)
        
triples = pd.DataFrame(triples)
triples.head(20)

In [ ]:
triples

## TODO: Try out MinIE and/or Allenai OpenIE-standalone

# Named Entity and Triple Matching

In [ ]:
entity_set = set(ner_dict.keys())
entity_set

#### Following needs to be optimized to reduce loss of knowledge

In [ ]:
final_triples = []
for row, col in triples.iterrows():
    col['subject'] = col['subject'].strip()

    # check if Named Entity in subject sentence fragment
    found_entity = False
    for named_entity in entity_set:
        if named_entity in col['subject']:
            col['subject'] = named_entity
            found_entity = True

    if found_entity:
        added = False
        entity2_sent = col['object']
        for entity in entity_set:
#             if entity in entity2_sent:
#                 final_triples.append(
#                     (ner_dict[col['subject']], col['subject'], col['relation'], 'O', col['object']))
#                 added = True
#         if not added:
            final_triples.append(('Node', col['subject'], col['relation'], 'Node', col['object']))

In [ ]:
final_triples

In [ ]:
final_df = pd.DataFrame(final_triples, columns=['Type1','Entity1','Relationship','Type2', 'Entity2']).drop_duplicates()
final_df
# final_df.to_csv('starwars1_processed.csv', encoding='utf-8', index=False)

> starwars1_processed Graph Visualization: https://graphcommons.com/graphs/01252887-9a23-4b33-a06d-02729a330beb

> Graph, using Stanford NER: https://graphcommons.com/graphs/1f5d76f7-69ad-4575-83dd-4c90afab7059

# Triple Linking

## Similarity with BERT

> Note this is limited due to average of word vectors

In [ ]:
import en_trf_bertbaseuncased_lg
nlp = en_trf_bertbaseuncased_lg.load()

In [ ]:
index = 0
for _,col1 in final_df.iterrows():
    head = col1['Entity2']
    doc1 = nlp(head)
    
    for _,col2 in final_df.iterrows():
        tail = col2['Entity2']
        if head == tail:
            continue
            
        doc2 = nlp(tail)
        confidence = doc1.similarity(doc2)
        
        if confidence > 0.85:   # 85% seems to work pretty well
            # Perform logic for linking
            new_tail = head if len(tail)<len(head) else tail
            
            col1['Entity2'] = new_tail
            col2['Entity2'] = new_tail

            print("Sentence 1:", doc1)
            print("Sentence 2:", doc2)
            print("Similarity:", confidence)
            index += 1
            print('Processed {} out of {}'.format(index,len(final_df)))
            print()

In [ ]:
final_df = final_df.drop_duplicates()
final_df

In [ ]:
final_df.to_csv('starwars_linked_processed.csv', encoding='utf-8', index=False)

> https://graphcommons.com/graphs/aae63b3e-870d-4c2e-83e4-bbca9f297b42

# Sentence Transformers

> https://github.com/UKPLab/sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer, util

In [4]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
# bert-base-nli-mean-tokens
# https://huggingface.co/sentence-transformers/bert-base-nli-mean-tokens

In [ ]:
%debug

In [5]:
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.4579
A man is playing guitar 		 A woman watches TV 		 Score: 0.1759
The new movie is awesome 		 The new movie is so great 		 Score: 0.9283


In [ ]:
index = 0
for _,col1 in final_df.iterrows():
    head = col1['Entity2']
    doc1 = nlp(head)
    
    for _,col2 in final_df.iterrows():
        tail = col2['Entity2']
        if head == tail:
            continue
            
        doc2 = nlp(tail)
        confidence = doc1.similarity(doc2)
        
        if confidence > 0.85:   # 85% seems to work pretty well
            # Perform logic for linking
            new_tail = head if len(tail)<len(head) else tail
            
            col1['Entity2'] = new_tail
            col2['Entity2'] = new_tail

            print("Sentence 1:", doc1)
            print("Sentence 2:", doc2)
            print("Similarity:", confidence)
            index += 1
            print('Processed {} out of {}'.format(index,len(final_df)))
            print()

# Textual Entailment with Keras_Parikh_Entailment

> https://github.com/explosion/spaCy/tree/master/examples/keras_parikh_entailment

In [ ]:
import en_vectors_web_lg
nlp = en_vectors_web_lg.load()

In [ ]:
final_triples

# Post Process & Entity Linking to existing Turtle Knowledge Bases

In [ ]:
import rdflib
graph = rdflib.Graph()
graph.parse('../data/starwars.ttl', format='turtle')

In [ ]:
query_str = """
    SELECT ?s ?p ?o
    WHERE {   
        ?s ?p ?o.
        #FILTER(?s="Darth Vader")
    }
    #LIMIT 10
"""
res = graph.query(query_str)

# for s,p,o in res:
#     print(s, '->', p, '->', o)